In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd

In [3]:
pw = pd.read_csv('/content/drive/MyDrive/reto optimizacion/MATRICES /cluster_12/cluster_12_duration.csv')
nodes = pd.read_csv('/content/drive/MyDrive/reto optimizacion/MATRICES /cluster_12/cluster_12_centroid_duration.csv')

In [4]:
pw

0       1       2       3       4       5       6
0     0.0  3943.0  4562.0  2967.0  2312.0  2755.0  2427.0
1  3119.0     0.0  5607.0  1798.0  1874.0  3800.0  2702.0
2  6067.0  6953.0     0.0  7258.0  5696.0  3404.0  7398.0
3  3158.0  1512.0  5646.0     0.0  2547.0  3839.0  2741.0
4  2686.0  2069.0  4766.0  3353.0     0.0  3136.0  3065.0
5  3390.0  4368.0  2992.0  4581.0  3111.0     0.0  3844.0
6  3024.0  4314.0  5727.0  3245.0  3264.0  3920.0     0.0

In [5]:
pw.rename(columns={'0': '1', '1': '2', '2': '3', '3': '4', '4': '5', '5': '6', '6': '7', '7': '8'
                    , '8': '9', '9': '10', '10': '11', '11': '12', '12': '13', '13': '14', '14': '15'
                    , '15': '16', '16': '17', '17': '18', '18': '19', '19': '20', '20': '21', '21': '22'
                    , '22': '23', '23': '24', '24': '25'}, inplace=True)

In [6]:
pw

1       2       3       4       5       6       7
0     0.0  3943.0  4562.0  2967.0  2312.0  2755.0  2427.0
1  3119.0     0.0  5607.0  1798.0  1874.0  3800.0  2702.0
2  6067.0  6953.0     0.0  7258.0  5696.0  3404.0  7398.0
3  3158.0  1512.0  5646.0     0.0  2547.0  3839.0  2741.0
4  2686.0  2069.0  4766.0  3353.0     0.0  3136.0  3065.0
5  3390.0  4368.0  2992.0  4581.0  3111.0     0.0  3844.0
6  3024.0  4314.0  5727.0  3245.0  3264.0  3920.0     0.0

In [7]:
pw.index = range(1, len(pw) + 1, 1)

In [8]:
nodes.rename(columns={"0":'d0'}, inplace = True)
# nodes.drop(7, axis=0, inplace=True)
nodes['demand'] = nodes['d0'] + 2400

nodes.reset_index(drop=True, inplace=True)
nodes.loc[-1] = [0, 0]
nodes.index = nodes.index + 1
nodes.sort_index(inplace=True)

nodes.head(18)

d0  demand
0     0.0     0.0
1  3727.0  6127.0
2  4772.0  7172.0
3  6346.0  8746.0
4  4811.0  7211.0
5  3972.0  6372.0
6  3459.0  5859.0
7  4892.0  7292.0

In [9]:
savings = dict()
for r in pw.index:
    #print(f'{r = }')
    for c in pw.columns:
        #print(f'{c = }')
        if int(c) != int(r):
            a = max(int(r), int(c))
            b = min(int(r), int(c))
            key = '(' + str(a) + ',' + str(b) + ')'
            savings[key] = nodes['d0'][int(r)] + nodes['d0'][int(c)] - pw[c][r]

# put savings in a pandas dataframe, and sort by descending
sv = pd.DataFrame.from_dict(savings, orient = 'index')
sv.rename(columns = {0:'saving'}, inplace = True)
sv.sort_values(by = ['saving'], ascending = False, inplace = True)
sv

saving
(4,2)  8071.0
(6,3)  6813.0
(5,2)  6675.0
(7,4)  6458.0
(7,5)  5600.0
(7,1)  5595.0
(5,3)  5552.0
(7,3)  5511.0
(4,3)  5511.0
(5,4)  5430.0
(2,1)  5380.0
(4,1)  5380.0
(7,2)  5350.0
(5,1)  5013.0
(7,6)  4431.0
(6,5)  4320.0
(3,2)  4165.0
(3,1)  4006.0
(6,2)  3863.0
(6,1)  3796.0
(6,4)  3689.0

In [10]:
# convert link string to link list to handle saving's key, i.e. str(10, 6) to (10, 6)
def get_node(link):
    link = link[1:]
    link = link[:-1]
    nodes = link.split(',')
    return [int(nodes[0]), int(nodes[1])]

In [11]:
# determine if a node is interior to a route
def interior(node, route):
    try:
        i = route.index(node)
        # adjacent to depot, not interior
        if i == 0 or i == (len(route) - 1):
            label = False
        else:
            label = True
    except:
        label = False

    return label

In [12]:
# merge two routes with a connection link
def merge(route0, route1, link):
    if route0.index(link[0]) != (len(route0) - 1):
        route0.reverse()

    if route1.index(link[1]) != 0:
        route1.reverse()

    return route0 + route1

In [13]:
# sum up to obtain the total passengers belonging to a route
def sum_cap(route):
    sum_cap = 0
    for node in route:
        sum_cap += nodes.demand[node]
    return sum_cap

In [14]:
# determine 4 things:
# 1. if the link in any route in routes -> determined by if count_in > 0
# 2. if yes, which node is in the route -> returned to node_sel
# 3. if yes, which route is the node belongs to -> returned to route id: i_route
# 4. are both of the nodes in the same route? -> overlap = 1, yes; otherwise, no
def which_route(link, routes):
    # assume nodes are not in any route
    node_sel = list()
    i_route = [-1, -1]
    count_in = 0

    for route in routes:
        for node in link:
            try:
                route.index(node)
                i_route[count_in] = routes.index(route)
                node_sel.append(node)
                count_in += 1
            except:
                pass

    if i_route[0] == i_route[1]:
        overlap = 1
    else:
        overlap = 0

    return node_sel, count_in, i_route, overlap

In [15]:
# create empty routes
routes = list()

# if there is any remaining customer to be served
remaining = True

# define capacity of the vehicle
cap = 25200

# record steps
step = 0

# get a list of nodes, excluding the depot
node_list = list(nodes.index)
node_list.remove(0)

# run through each link in the saving list
for link in sv.index:
    step += 1
    if remaining:

        print('step ', step, ':')

        link = get_node(link)
        node_sel, num_in, i_route, overlap = which_route(link, routes)

        # condition a. Either, neither i nor j have already been assigned to a route,
        # ...in which case a new route is initiated including both i and j.
        if num_in == 0:
            if sum_cap(link) <= cap:
                routes.append(link)
                node_list.remove(link[0])
                node_list.remove(link[1])
                print('\t','Link ', link, ' fulfills criteria a), so it is created as a new route')
            else:
                print('\t','Though Link ', link, ' fulfills criteria a), it exceeds maximum load, so skip this link.')

        # condition b. Or, exactly one of the two nodes (i or j) has already been included
        # ...in an existing route and that point is not interior to that route
        # ...(a point is interior to a route if it is not adjacent to the depot D in the order of traversal of nodes),
        # ...in which case the link (i, j) is added to that same route.
        elif num_in == 1:
            n_sel = node_sel[0]
            i_rt = i_route[0]
            position = routes[i_rt].index(n_sel)
            link_temp = link.copy()
            link_temp.remove(n_sel)
            node = link_temp[0]

            cond1 = (not interior(n_sel, routes[i_rt]))
            cond2 = (sum_cap(routes[i_rt] + [node]) <= cap)

            if cond1:
                if cond2:
                    print('\t','Link ', link, ' fulfills criteria b), so a new node is added to route ', routes[i_rt], '.')
                    if position == 0:
                        routes[i_rt].insert(0, node)
                    else:
                        routes[i_rt].append(node)
                    node_list.remove(node)
                else:
                    print('\t','Though Link ', link, ' fulfills criteria b), it exceeds maximum load, so skip this link.')
                    continue
            else:
                print('\t','For Link ', link, ', node ', n_sel, ' is interior to route ', routes[i_rt], ', so skip this link')
                continue

        # condition c. Or, both i and j have already been included in two different existing routes
        # ...and neither point is interior to its route, in which case the two routes are merged.
        else:
            if overlap == 0:
                cond1 = (not interior(node_sel[0], routes[i_route[0]]))
                cond2 = (not interior(node_sel[1], routes[i_route[1]]))
                cond3 = (sum_cap(routes[i_route[0]] + routes[i_route[1]]) <= cap)

                if cond1 and cond2:
                    if cond3:
                        route_temp = merge(routes[i_route[0]], routes[i_route[1]], node_sel)
                        temp1 = routes[i_route[0]]
                        temp2 = routes[i_route[1]]
                        routes.remove(temp1)
                        routes.remove(temp2)
                        routes.append(route_temp)
                        try:
                            node_list.remove(link[0])
                            node_list.remove(link[1])
                        except:
                            #print('\t', f"Node {link[0]} or {link[1]} has been removed in a previous step.")
                            pass
                        print('\t','Link ', link, ' fulfills criteria c), so route ', temp1, ' and route ', temp2, ' are merged')
                    else:
                        print('\t','Though Link ', link, ' fulfills criteria c), it exceeds maximum load, so skip this link.')
                        continue
                else:
                    print('\t','For link ', link, ', Two nodes are found in two different routes, but not all the nodes fulfill interior requirement, so skip this link')
                    continue
            else:
                print('\t','Link ', link, ' is already included in the routes')
                continue

        for route in routes:
            print('\t','route: ', route, ' with load ', sum_cap(route))
    else:
        print('-------')
        print('All nodes are included in the routes, algorithm closed')
        break

    remaining = bool(len(node_list) > 0)

# check if any node is left, assign to a unique route
for node_o in node_list:
    routes.append([node_o])

# add depot to the routes
for route in routes:
    route.insert(0,0)
    route.append(0)

print('------')
print('Routes found are: ')

routes

step  1 :
	 Link  [4, 2]  fulfills criteria a), so it is created as a new route
	 route:  [4, 2]  with load  14383.0
step  2 :
	 Link  [6, 3]  fulfills criteria a), so it is created as a new route
	 route:  [4, 2]  with load  14383.0
	 route:  [6, 3]  with load  14605.0
step  3 :
	 Link  [5, 2]  fulfills criteria b), so a new node is added to route  [4, 2] .
	 route:  [4, 2, 5]  with load  20755.0
	 route:  [6, 3]  with load  14605.0
step  4 :
	 Though Link  [7, 4]  fulfills criteria b), it exceeds maximum load, so skip this link.
step  5 :
	 Though Link  [7, 5]  fulfills criteria b), it exceeds maximum load, so skip this link.
step  6 :
	 Link  [7, 1]  fulfills criteria a), so it is created as a new route
	 route:  [4, 2, 5]  with load  20755.0
	 route:  [6, 3]  with load  14605.0
	 route:  [7, 1]  with load  13419.0
-------
All nodes are included in the routes, algorithm closed
------
Routes found are: 


[[0, 4, 2, 5, 0], [0, 6, 3, 0], [0, 7, 1, 0]]

Routes found are:

[0, 4, 2, 5, 0],

 [0, 6, 3, 0],

  [0, 7, 1, 0]